In [ ]:
### General Imports
import pandas as pd
import tensorflow as tf
import numpy as np

dataframe = pd.read_csv("Dataset-Mental-Disorders.csv")
dataframe = dataframe.drop(["Patient Number"], axis = 1)

### Data Preprocessing 
for row_number in range(len(dataframe)):

  # Columns 0 -> 3: ['Sometimes', 'Usually', 'Seldom', 'Most-Often']
  columns_type1 = dataframe.columns[:4]
  for column in columns_type1:
    value = dataframe.loc[row_number, column]
    if value == 'Sometimes': dataframe.loc[row_number, column] = 0.0
    elif value == 'Usually': dataframe.loc[row_number, column] = 1.0
    elif value == 'Seldom': dataframe.loc[row_number, column] = 2.0
    else: dataframe.loc[row_number, column] = 3.0


  # Columns 4 -> 13: ['YES', 'NO']
  columns_type2 = dataframe.columns[4:14]
  for column in columns_type2:
    value = dataframe.loc[row_number, column]
    if value == 'NO': dataframe.loc[row_number, column] = 0.0
    else: dataframe.loc[row_number, column] = 1.0

  # Columns 14 -> 16: [int from int] => get the first int
  columns_type3 = dataframe.columns[14: 17]
  for column in columns_type3:
    value = dataframe.loc[row_number, column].split()
    dataframe.loc[row_number, column] = float(value[0]) 
  
  value = dataframe.loc[row_number, "Expert Diagnose"]
  if value == "Bipolar Type-2": dataframe.loc[row_number, "Expert Diagnose"] = 0
  elif value == "Depression": dataframe.loc[row_number, "Expert Diagnose"] = 1
  elif value == "Bipolar Type-1": dataframe.loc[row_number, "Expert Diagnose"] = 2
  elif value == "Normal": dataframe.loc[row_number, "Expert Diagnose"] = 3

In [ ]:
### Change Data Type of features to make them compatible with tensors
### Split Data
features = dataframe.iloc[:, :-1].values
features = np.asarray(features).astype('float32')
features = tf.constant(features)  

label = dataframe.iloc[:, -1].values
label = np.asarray(label).astype('float32')
label = tf.constant(label)

In [ ]:
### Normalize the Tensor
from keras.layers import Normalization
normalizer = Normalization()
normalizer.adapt(features)

In [ ]:
### Build ANN
from keras import Sequential
from keras.layers import Dense, InputLayer
from keras.optimizers import Adam
 
ANN = Sequential()
ANN.add( InputLayer(shape = (17,)) )
ANN.add( Dense(units = 17, activation = "relu") )
ANN.add( Dense(units = 17, activation = "relu") )
ANN.add( Dense(units = 1, activation = "softmax") )

### Train ANN
from keras.metrics import Accuracy
from keras.losses import CategoricalCrossentropy
ANN.compile( optimizer = Adam(), loss = "sparse_categorical_crossentropy", metrics = [Accuracy()] )
ANN.fit(features, label, epochs = 100)